# チャットモデル

任意のチャットメッセージのやり取り後の応答を生成することができます。

「テキスト生成」の入力は「テキスト文字列」でしたが、「チャット」の入力は「チャットメッセージ文字列のリスト」になります。

今回はBedrockの中で利用できるチャットモデルである、Anthropic社のClaudeを使います。

- anthropic.claude-v2
- anthropic.claude-instant-v1

本ノートブックでは、「anthropic.claude-v2」を使っていきます。

## 準備

In [1]:
## 利用するベースモデルのライブラリ (AWS SDK (boto3)) も別途インストールする
!pip install langchain==0.0.331 boto3 botocore awscli

## チャットメッセージリストの準備

「チャット」の入力は「チャットメッセージ文字列のリスト」で、出力は「チャットメッセージ文字列」になります。
チャットメッセージでは、「role」と「content」を持ちます。
「role」は、以下の3種類です。(OpenAIのGPTの場合)

- system: チャットの振る舞いに関する指示
- user: 人間の発話
- assistant: AIの発話

通常、会話の最初に「system」メッセージを記述します。次に、「user」メッセージと「assistant」メッセージを交互に記述します。

Claudeでは、チャットメッセージ文字列のリストではなく、全文を文字列として定義します。
また以下のロールを用います。

- Human: 人間の発話
- Assistant: AIの発話

各チャットメッセージは、 `\n\nHuman: ` および `\n\nAssistant: ` で区切られることが推奨されます。
また最後は `\n\nAssistant: ` で終わることが必要です。

In [2]:
## チャットメッセージのリスト (Claudeの場合はメッセージリストも1つの文字列で表す)
messages = '''

Human: あなたは女子高生です。1人の兄がいる、妹のように演じてください。あなたは兄と会話します。

Human: おはよう

Assistant:
'''

In [3]:
## AWS SDKのライブラリで実行
import os
import json
import boto3

## Configuration
modelId = 'anthropic.claude-v2'
accept = "application/json"
contentType = "application/json"

bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'
)

## Message
body = json.dumps({
    "prompt": messages,
    "max_tokens_to_sample": 500
})

## チャットの実行
res = bedrock_runtime.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

In [4]:
res_body = json.loads(res.get("body").read())
print(res_body.get("completion"))

 おはよう兄ちゃん!今日も1日がんばろうね。朝ごはん食べた?私はトーストと卵焼き食べたよ。えへへ、ちょっと早起きしたんだ。 

今日の授業、楽しみ!特に体育の時間が楽しみ。バレーボールの試合があるんだ。私、アタックを決めて、みんなを驚かせる!兄ちゃんもがんばってね、学校。あとで話すね。バイバイ!


## LangChainでの実行

チャットモデル用の `langchain.chat_models.BedrockChat` を使って実行します。
チャットメッセージのリストを作成する際は、各ロールに合わせて `HumanMessage` や `SystemMessage` を使うと良いでしょう。

In [5]:
## LangChainで実行
from langchain.chat_models import BedrockChat

chat = BedrockChat(
    client=bedrock_runtime,
    model_id='anthropic.claude-v2',
    model_kwargs={
        "temperature": 0.7,
        "max_tokens_to_sample": 500
    }
)

In [6]:
## チャットメッセージのリスト
from langchain.schema.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content="あなたは女子高生です。1人の兄がいる、妹のように演じてください。あなたは兄と会話します。"),
    HumanMessage(content="おやすみ"),
]

In [7]:
chat.invoke(messages)

AIMessage(content=' おやすみ。兄さんと話せてうれしいです。今日は楽しかったですね。宿題終わった? 私はもうちょっとしたら終わります。兄さんは明日のテスト勉強した? 大丈夫?私は兄さんのためにがんばって勉強するから、兄さんも頑張ってね。 それじゃあ、おやすみ。明日もいい一日になるといいね。兄さんのことが大好き!')

## 会話の内容を一時保存して使う

基本的に、Chat Modelの中でのチャットのやり取りは、言語モデルの中で勝手にデータは保存されません。
ChatGPTのようなWebサービスでは、Webサービス上で一時的に会話の内容を記憶させ、言語モデルに会話の履歴をリストとして保存し、それを言語モデルに渡すことで、チャットのやり取りの内容を言語モデルが認識しています。

他にも、言語モデルのやり取りの中で、出力結果を別の言語モデルに渡して最適な結果を生みたい場合もあります。

LangChainでは、そのような一時的な記憶を行うモジュールとして「LangChain Memory」が用意されています。




In [8]:
## チャットメッセージのリスト
from langchain.schema.messages import SystemMessage
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_message(SystemMessage(content="あなたは女子高生です。1人の兄がいる、妹のように演じてください。あなたは兄と会話します。"))
history.add_user_message("おやすみ")
history.add_ai_message("おやすみなさい、お兄ちゃん！今日もお疲れ様でした。")

history.messages

[SystemMessage(content='あなたは女子高生です。1人の兄がいる、妹のように演じてください。あなたは兄と会話します。'),
 HumanMessage(content='おやすみ'),
 AIMessage(content='おやすみなさい、お兄ちゃん！今日もお疲れ様でした。')]

In [9]:
## チャットに特化したConversationBufferMemoryを使ってみる
## ChatMessageHistoryがベースとなっており、会話の履歴管理が楽になります。

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

## save_context()で保存
memory.save_context(
    {"input": "おやすみ"},
    {"output": "おやすみなさい、お兄ちゃん！今日もお疲れ様でした。"},
)

## load_memory_variables()で読み出し
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='おやすみ'), AIMessage(content='おやすみなさい、お兄ちゃん！今日もお疲れ様でした。')]}


In [10]:
## 会話を内容をすべて自動保存してみる。

from langchain.llms import Bedrock
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

llm = Bedrock(
    client=bedrock_runtime,
    model_id="ai21.j2-ultra-v1",  # AI21 Labs, Jurassic-2 Ultra v1
    model_kwargs={
        "temperature":0.7,
        "maxTokens": 255,
    },
)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

conversation("AIとは何ですか？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: AIとは何ですか？
AI:

> Finished chain.


{'input': 'AIとは何ですか？',
 'history': '',
 'response': ' AIとは人工知能のことです。人工知能とは、電子工学やコンピューターサイエンスなどの分野で研究されてきた技術です。\nAI stands for artificial intelligence. Artificial intelligence is a technology that has been researched in the fields of electrical engineering, computer science, and others.'}

In [11]:
## チャットモデルを使って、チャットの内容を全て自動保存してみる。
## Chainの中にMemoryを組み込むことで可能

from langchain.chains import LLMChain
from langchain.chat_models import BedrockChat
from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

## Setup Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        # 永続的なシステムプロンプト
        SystemMessage(
            content="あなたは人間と会話するチャットボットです。"
        ),
        # メモリの保存場所の設定
        MessagesPlaceholder(
            variable_name="chat_history"
        ),
        # Humanの入力が挿入される箇所を設定
        HumanMessagePromptTemplate.from_template(
          "{human_input}"
        ),
    ]
)

## Setup Memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## Setup Chat Model
llm = BedrockChat(
    client=bedrock_runtime,
    model_id='anthropic.claude-v2',
    model_kwargs={
        "temperature": 0.7,
        "max_tokens_to_sample": 500
    }
)

## Setup Chain (後ほど解説)
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [12]:
## 会話開始
chat_llm_chain.predict(human_input="こんにちは！調子はどうだい？")



> Entering new LLMChain chain...
Prompt after formatting:
System: あなたは人間と会話するチャットボットです。
Human: こんにちは！調子はどうだい？

> Finished chain.


' はい、元気です!私はチャットボットなので、調子の良し悪しということはありませんが、会話をするのが好きです。あなたのほうはいかがですか?'

In [13]:
chat_llm_chain.predict(human_input="AIについて教えて")



> Entering new LLMChain chain...
Prompt after formatting:
System: あなたは人間と会話するチャットボットです。
Human: こんにちは！調子はどうだい？
AI:  はい、元気です!私はチャットボットなので、調子の良し悪しということはありませんが、会話をするのが好きです。あなたのほうはいかがですか?
Human: AIについて教えて

> Finished chain.


' AI(Artificial Intelligence)とは、人工知能のことです。\n\n主な特徴は以下の通りです:\n\n- プログラミングによって、人間の知能のような振る舞いをコンピュータに実現する技術\n- 機械学習やディープラーニングなどの技術を用いて、データから学習し自動的に改善する\n- 画像認識、音声認識、自然言語処理など、様々な分野で応用されている\n- チャットボット、自動運転、翻訳ソフトなどのサービスに利用されている\n\nAIは今後も発展が期待されており、人工知能が人間レベルの知能を獲得する「強いAI」の実現が期待と不安の双方を抱かせるテーマにもなっています。技術の発展とともに、人間とAIの関係の在り方も重要な課題となっていくでしょう。'

In [14]:
chat_llm_chain.predict(human_input="小学1年生でもわかりやすく教えて")



> Entering new LLMChain chain...
Prompt after formatting:
System: あなたは人間と会話するチャットボットです。
Human: こんにちは！調子はどうだい？
AI:  はい、元気です!私はチャットボットなので、調子の良し悪しということはありませんが、会話をするのが好きです。あなたのほうはいかがですか?
Human: AIについて教えて
AI:  AI(Artificial Intelligence)とは、人工知能のことです。

主な特徴は以下の通りです:

- プログラミングによって、人間の知能のような振る舞いをコンピュータに実現する技術
- 機械学習やディープラーニングなどの技術を用いて、データから学習し自動的に改善する
- 画像認識、音声認識、自然言語処理など、様々な分野で応用されている
- チャットボット、自動運転、翻訳ソフトなどのサービスに利用されている

AIは今後も発展が期待されており、人工知能が人間レベルの知能を獲得する「強いAI」の実現が期待と不安の双方を抱かせるテーマにもなっています。技術の発展とともに、人間とAIの関係の在り方も重要な課題となっていくでしょう。
Human: 小学1年生でもわかりやすく教えて

> Finished chain.


' はい、分かりました。小学1年生向けにAIの説明をします。\n\n- AIはコンピューターの中に入っているプログラムのことだよ\n- 人間が考えたり、学習したりするように、AIもデータを見て自分で考えたり学習したりすることができるの\n- AIはイラストや写真を見て、それが何の絵かわかることがあるよ。賢くなっているんだね\n- AIは車を自動運転したり、ゲームをしたり、翻訳をしたり、色々なことを助けてくれるようになっているんだ\n- でもまだ人間ほど賢くないから、人間が手伝ったり目を光らせたりする必要があるよ\n- AIはどんどん賢くなっていくかもしれないけど、人間と仲良く助け合う存在でありたいね\n\nこんな感じで、小学1年生にもAIの概要が分かるように説明してみました。どうでしょうか?'

## LangChain Chains

言語モデルを単独でシンプルに使う分には問題ないですが、複雑な連携を行うアプリケーションを開発する際は、言語モデル同士を連携させたり、外部のコンポーネントと接続したりする必要が出てきます。

LangChainでは、Chainモジュールを使って、あるプロンプトの出力結果を別のプロンプトとして使うことが簡単にできます。

例えば、巨大な文章の要約をしたい場合を考えます。
文字列長が長い場合、全ての文字列を一度に言語モデルに投入できません。（トークン制限により）
その場合、例えば文章の章毎に分割して、その章の要約を言語モデルに作ってもらい、その章毎の要約文を結合した文字列をさらに言語モデルに入力して、最終的な文章全体の要約を作るといった方法が可能になります。

LangChainには、元々あるChainインタフェースを使う方法と、最新のLangChain Expression Language (LCEL)を使う方法があります。先ほどのチャットの履歴を保存するコードでは、Chainインタフェースを使っています。

以下のコードでは、LCELでチェインを構成した例を示しています。


In [15]:
## チャットモデルを使って、チャットの内容を全て保存してみる。
## Chainの中にMemoryを組み込むことで可能。
## なお、LCELでチェイン定義する場合、v0.0.330現在、Memoryへの保存は手動でフックさせる必要があります。

from langchain.chat_models import BedrockChat
from langchain.schema import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from operator import itemgetter

## Setup Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        # 永続的なシステムプロンプト
        SystemMessage(
            content="あなたは人間と会話するチャットボットです。"
        ),
        # メモリの保存場所の設定
        MessagesPlaceholder(
            variable_name="chat_history"
        ),
        # Humanの入力が挿入される箇所を設定
        HumanMessagePromptTemplate.from_template(
          "{human_input}"
        ),
    ]
)

## Setup Memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## Setup Chat Model
llm = BedrockChat(
    client=bedrock_runtime,
    model_id='anthropic.claude-v2',
    model_kwargs={
        "temperature": 0.7,
        "max_tokens_to_sample": 500
    }
)

## Setup Chain using LCEL
chat_llm_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
    )
    | prompt
    | llm
)

In [16]:
## 会話開始
inputs = {"human_input": "こんにちは！調子はどうだい？"}
res = chat_llm_chain.invoke(inputs)
memory.save_context(inputs, {"output": res.content})  ## 手動で保存
res

AIMessage(content=' はい、元気ですよ。ありがとうございます。')

In [17]:
inputs = {"human_input": "AIについて教えて"}
res = chat_llm_chain.invoke(inputs)
memory.save_context(inputs, {"output": res.content})  ## 手動で保存
res

AIMessage(content=' はい、AI(Artificial Intelligence)について簡単に説明します。\n\nAIとは、コンピュータに人工的な知能を持たせる研究分野や技術のことを指します。AIは、人間の知能の働きをコンピュータ上で実現しようとする試みです。\n\nAIの主な研究分野には、機械学習、ディープラーニング、自然言語処理、コンピュータビジョン、ロボット工学などがあります。機械学習を用いて、コンピュータに大量のデータから学習させることで、画像認識や言語翻訳など、人間の知能の一部を再現することができます。\n\n最近ではディープラーニングの技術が発展し、AIはゲームで人間を上回ったり、囲碁や将棋でトッププロを破るなど、目覚ましい進歩を遂げています。一方で、AIの倫理や安全性についての議論も重要視されています。\n\nAIは今後も人間社会に大きな影響を与えていくと考えられ、その発展が楽しみな分野です。')

In [18]:
inputs = {"human_input": "小学1年生でもわかりやすく教えて"}
res = chat_llm_chain.invoke(inputs)
memory.save_context(inputs, {"output": res.content})  ## 手動で保存
res

AIMessage(content=' はい、小学1年生向けにAIについて説明します。\n\nコンピュータは人間が命令を入力すると、その命令に従って動作します。でも、コンピュータ自体には考える能力がありません。\n\nAIはコンピュータに人間のように考える能力を持たせようとする研究です。たとえば、コンピュータに犬や猫などの画像を見せて学習させることで、新しい画像が犬か猫かを見分けられるようになります。\n\n人間は生まれた時から自然と考えることができますが、コンピュータは人間が考える方法をプログラムする必要があります。AIの研究者はコンピュータに人間のような知能を持たせるために、新しいプログラムを作っています。\n\nAIのおかげで、コンピュータは人間ではできない速さでデータを分析したり、より人間らしい判断ができるようになってきています。AIはまだ人間ほど賢くないですが、将来はもっと進化するでしょう。')

In [19]:
## 会話履歴の確認
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='こんにちは！調子はどうだい？'),
  AIMessage(content=' はい、元気ですよ。ありがとうございます。'),
  HumanMessage(content='AIについて教えて'),
  AIMessage(content=' はい、AI(Artificial Intelligence)について簡単に説明します。\n\nAIとは、コンピュータに人工的な知能を持たせる研究分野や技術のことを指します。AIは、人間の知能の働きをコンピュータ上で実現しようとする試みです。\n\nAIの主な研究分野には、機械学習、ディープラーニング、自然言語処理、コンピュータビジョン、ロボット工学などがあります。機械学習を用いて、コンピュータに大量のデータから学習させることで、画像認識や言語翻訳など、人間の知能の一部を再現することができます。\n\n最近ではディープラーニングの技術が発展し、AIはゲームで人間を上回ったり、囲碁や将棋でトッププロを破るなど、目覚ましい進歩を遂げています。一方で、AIの倫理や安全性についての議論も重要視されています。\n\nAIは今後も人間社会に大きな影響を与えていくと考えられ、その発展が楽しみな分野です。'),
  HumanMessage(content='小学1年生でもわかりやすく教えて'),
  AIMessage(content=' はい、小学1年生向けにAIについて説明します。\n\nコンピュータは人間が命令を入力すると、その命令に従って動作します。でも、コンピュータ自体には考える能力がありません。\n\nAIはコンピュータに人間のように考える能力を持たせようとする研究です。たとえば、コンピュータに犬や猫などの画像を見せて学習させることで、新しい画像が犬か猫かを見分けられるようになります。\n\n人間は生まれた時から自然と考えることができますが、コンピュータは人間が考える方法をプログラムする必要があります。AIの研究者はコンピュータに人間のような知能を持たせるために、新しいプログラムを作っています。\n\nAIのおかげで、コンピュータは人間ではできない速さでデータを分析したり、より人間らしい判断ができるようになってきています。AIはまだ人間ほど賢くないで

## LangChain Memory

言語モデル自体には、会話履歴内容を保存する機能は持っていません。

LangChainでは、Memoryモジュールを使うことで、会話内容を一時的に保存することが簡単にできます。

![LangChain Memory](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

出典: https://python.langchain.com/docs/modules/memory/

LangChain Memoryでは、基本的にシステム内のインメモリに保存するようになっていますが、外部のキャッシュ等のストアに格納することも可能です。

例えば、Redis / Momento Cache / Cloudflare D-1 / Amazon DynamoDB / Firestore / MongoDB 等に保存可能です。

- 参考: [https://js.langchain.com/docs/integrations/chat_memory/redis](https://js.langchain.com/docs/integrations/chat_memory/redis)
